In [1]:
# -*- coding: utf-8 -*-
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K

Using TensorFlow backend.


In [2]:
#EPISODES = 5000
EPISODES = 100

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    def _huber_loss(self, target, prediction):
        # sqrt(1+error^2)-1
        error = prediction - target
        return K.mean(K.sqrt(1+K.square(error))-1, axis=-1)

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss=self._huber_loss,
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def update_target_model(self):
        # copy weights from model to target_model
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state)
            if done:
                target[0][action] = reward
            else:
                # a = self.model.predict(next_state)[0]
                t = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * np.amax(t)
                # target[0][action] = reward + self.gamma * t[np.argmax(a)]
            self.model.fit(state, target, nb_epoch=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)




In [3]:
if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    # agent.load("./save/cartpole-ddqn.h5")
    done = False
    batch_size = 32

    for e in range(EPISODES):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for time in range(500):
            # env.render()
            action = agent.act(state)
            debug = False
            if debug:
                print(action)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            debug=False
            if debug:
                print('next_state')
                print(next_state)
                print(state_size)
            next_state = np.reshape(next_state, [1, state_size])
            if debug:
                print('reshaped next_state')
                print(next_state)
                print('=========')
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                agent.update_target_model()
                print("episode: {}/{}, score: {}, e: {:.2}"
                      .format(e, EPISODES, time, agent.epsilon))
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)
        # if e % 10 == 0:
        #     agent.save("./save/cartpole-ddqn.h5")


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
episode: 0/100, score: 15, e: 1.0
episode: 1/100, score: 9, e: 1.0
episode: 2/100, score: 11, e: 0.95
episode: 3/100, score: 27, e: 0.72
episode: 4/100, score: 28, e: 0.55
episode: 5/100, score: 8, e: 0.5
episode: 6/100, score: 22, e: 0.4
episode: 7/100, score: 16, e: 0.34
episode: 8/100, score: 21, e: 0.28
episode: 9/100, score: 8, e: 0.26
episode: 10/100, score: 9, e: 0.24
episode: 11/100, score: 9, e: 0.21
episode: 12/100, score: 10, e: 0.19
episode: 13/100, score: 16, e: 0.17
episode: 14/100, score: 21, e: 0.13
episode: 15/100, score: 28, e: 0.1
episode: 16/100, score: 26, e: 0.078
episode: 17/100, score: 37, e: 0.054
episode: 18/100, score: 24, e: 0.042
episode: 19/100, score: 16, e: 0.036
episode: 20/100, score: 27, e: 0.027
episode: 21/100, score: 42, e: 0.018
episode: 22/100, score: 10, e: 0.016
episode: 23/100, score: 10, e: 0.015
episode: 24/100, score: 9, e: 0.013
episode: 25/1